In [1]:
categories = {25:
        {
            "id": 1,
            "name": "catether",
            "supercategory": ""
        },
        0: {
            "id": 2,
            "name": "1",
            "supercategory": ""
        },
        16:{
            "id": 3,
            "name": "2",
            "supercategory": ""
        },
        17:{
            "id": 4,
            "name": "3",
            "supercategory": ""
        },
        18:{
            "id": 5,
            "name": "4",
            "supercategory": ""
        },
        19:{
            "id": 6,
            "name": "5",
            "supercategory": ""
        },
        20:{
            "id": 7,
            "name": "6",
            "supercategory": ""
        },
        21:{
            "id": 8,
            "name": "7",
            "supercategory": ""
        },
        22:{
            "id": 9,
            "name": "8",
            "supercategory": ""
        },
        23:{
            "id": 10,
            "name": "9",
            "supercategory": ""
        },
        24:{
            "id": 11,
            "name": "9a",
            "supercategory": ""
        },
        1:{
            "id": 12,
            "name": "10",
            "supercategory": ""
        },
        2:{
            "id": 13,
            "name": "10a",
            "supercategory": ""
        },
        3:{
            "id": 14,
            "name": "11",
            "supercategory": ""
        },
        4:{
            "id": 15,
            "name": "12",
            "supercategory": ""
        },
        5:{
            "id": 16,
            "name": "12a",
            "supercategory": ""
        },
        7:{
            "id": 17,
            "name": "13",
            "supercategory": ""
        },
        8:{
            "id": 18,
            "name": "14",
            "supercategory": ""
        },
        9:{
            "id": 19,
            "name": "14a",
            "supercategory": ""
        },
       11:{
            "id": 20,
            "name": "15",
            "supercategory": ""
        },
        12:{
            "id": 21,
            "name": "16",
            "supercategory": ""
        },
        13:{
            "id": 22,
            "name": "16a",
            "supercategory": ""
        },
        14:{
            "id": 23,
            "name": "16b",
            "supercategory": ""
        },
        15:{
            "id": 24,
            "name": "16c",
            "supercategory": ""
        },
        6:{
            "id": 25,
            "name": "12b",
            "supercategory": ""
        },
        10:{
            "id": 26,
            "name": "14b",
            "supercategory": ""
        }
}

In [2]:
import numpy as np

In [3]:
import json
import os
from collections import defaultdict

def yolo_to_coco_seg(yolo_path, save_path, categories=defaultdict(dict)):
    """
    Convert YOLOv8 segmentation dataset to COCO segmentation dataset format.
    
    Args:
        yolo_path (str): Path to the directory containing Pascal VOC annotations.
        save_path (str): Path to save COCO formatted annotations.
    """
    annotations = []
    images = []
    annotation_id = 1
    
    # Get list of annotation files
    annotation_files = [f for f in os.listdir(yolo_path) if f.endswith('.txt')]

    
    annotations = []
    images = []
    annotation_id = 1

    # Loop through annotation files
    for i, annotation_file in enumerate(annotation_files):
        # print(annotation_file)
        width = 512
        height = 512
        # print(annotation_file)

        with open(os.path.join(yolo_path, annotation_file)) as f:
            anns = f.readlines()
        # Add image information to images list
        image_id = i
        images.append({'id': image_id, 'width': width, 'height': height, 'file_name': annotation_file[:-4]+'.jpg'})
        
        # Loop through object tags
        for line in anns:
            
            # Get object name and bounding box coordinates
            l = line.split()
            seg = np.array(l[1:], dtype=np.float16)
            mul = np.array([width, height]*(len(seg)//2))
            seg = list(seg * mul) 
            if len(seg) == 0:
                continue
            # print(seg)
            x = seg[::2]
            y = seg[1::2]
            id = l[0]
            if id == '26':
                continue
            # Add annotation information to annotations list
            annotation = {'id': annotation_id, 'image_id': image_id, 'category_id': categories[int(id)]['id'],
                        'segmentation': [seg], 'area': (max(x)-min(x)) * (max(y)-min(y)), 'bbox': [min(x), min(y), (max(x)-min(x)), (max(y)-min(y))],
                        'iscrowd': 0}
            annotations.append(annotation)
            
            annotation_id += 1

    # Create COCO dictionary
    coco_dict = {'images': images, 'categories': list(categories.values()), 'annotations': annotations}
    
    # Save COCO dictionary as JSON file
    with open(os.path.join(save_path, 'annotations.json'), 'w') as f:
        json.dump(coco_dict, f)


In [4]:
yolo_to_coco_seg(r"G:\PROGRAMMIN PROJECTS\semester 7\cmc\data\cat_coco\cat_ves\yolo\dicoms\batch2\labels", 
                r"G:\PROGRAMMIN PROJECTS\semester 7\cmc\data\cat_coco\cat_ves\yolo\dicoms\batch2",
                categories)